<a href="https://colab.research.google.com/github/ndb796/Captcha-Experiment/blob/master/colab/Captcha%20Experiment%20(CNN%20Basic%2C%20Gray).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install captcha

In [0]:
''' Captcha Setting '''

import os
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np


# character_set = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
character_set = '0123456789'
character_set_length = len(character_set)

number_per_image = 4
width = 40 + 20 * number_per_image
height = 100

train_dataset_path = 'dataset' + os.path.sep + 'train'
test_dataset_path = 'dataset' + os.path.sep + 'test'

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
])


class Dataset():
    def __init__(self, folder, transform=None):
        self.train_image_file_paths = [os.path.join(folder, image_file) for image_file in os.listdir(folder)]
        self.transform = transform

    def __len__(self):
        return len(self.train_image_file_paths)

    def __getitem__(self, idx):
        image_root = self.train_image_file_paths[idx]
        image_name = image_root.split(os.path.sep)[-1]
        image = Image.open(image_root)
        if self.transform is not None:
            image = self.transform(image)
        label = encode(image_name.split('_')[0])
        return image, label


def get_train_data_loader():
    dataset = Dataset(train_dataset_path, transform=transform)
    return DataLoader(dataset, batch_size=64, shuffle=True)


def get_test_data_loader():
    dataset = Dataset(test_dataset_path, transform=transform)
    return DataLoader(dataset, batch_size=1, shuffle=True)


def char2pos(c):
    if c == '_':
        k = 62
        return k
    k = ord(c) - 48
    if k > 9:
        k = ord(c) - 65 + 10
        if k > 35:
            k = ord(c) - 97 + 26 + 10
            if k > 61:
                raise ValueError('error')
    return k


def encode(text):
    vector = np.zeros(character_set_length * number_per_image, dtype=float)
    for i, c in enumerate(text):
        idx = i * character_set_length + char2pos(c)
        vector[idx] = 1.0
    return vector


def decode(vec):
    char_pos = vec.nonzero()[0]
    text = []
    for i, c in enumerate(char_pos):
        char_idx = c % character_set_length
        if char_idx < 10:
            char_code = char_idx + ord('0')
        elif char_idx < 36:
            char_code = char_idx - 10 + ord('A')
        elif char_idx < 62:
            char_code = char_idx - 36 + ord('a')
        elif char_idx == 62:
            char_code = ord('_')
        else:
            raise ValueError('error')
        text.append(chr(char_code))
    return "".join(text)


e = encode("8937")
print(e)
print(decode(e))

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
8937


In [0]:
''' Captcha Generator '''

from captcha.image import ImageCaptcha
import itertools
import os
import shutil
import uuid


def generate_captcha(directory, characters, n):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    if not os.path.exists(directory):
        os.makedirs(directory)

    image = ImageCaptcha(width=width, height=height)
    for count in range(1, n + 1):
        print('Generating: (' + str(count) + '/' + str(n) + ')')
        for i in itertools.permutations(characters, number_per_image):
            captcha = ''.join(i)
            file_name = directory + '/' + captcha + '_' + str(uuid.uuid4()) + '.png'
            image.write(captcha, file_name)


# Execute
generate_captcha(train_dataset_path, character_set, 6) # n = 3: 70%, n = 6: 
generate_captcha(test_dataset_path, character_set, 1)

Generating: (1/6)
Generating: (2/6)
Generating: (3/6)
Generating: (4/6)
Generating: (5/6)
Generating: (6/6)
Generating: (1/1)


In [0]:
''' Captcha Model '''

import torch.nn as nn


# CNN Model (2 Convolution Layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.Dropout(0.5),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.Dropout(0.5),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.Dropout(0.5),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Sequential(
            nn.Linear((width // 8) * (height // 8) * 64, 1024),
            nn.Dropout(0.5),  # drop 50% of the neuron
            nn.ReLU())
        self.rfc = nn.Sequential(
            nn.Linear(1024, number_per_image * character_set_length),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.rfc(out)
        return out

In [0]:
''' Captcha Train '''

import torch
import torch.nn as nn
from torch.autograd import Variable


# Hyper Parameters
num_epochs = 30
batch_size = 100
learning_rate = 0.001


def main():
    # GPU Setting
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('Cuda Available')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    cnn = CNN()
    cnn.to(device)
    cnn.train()
    
    print('Model Initialization')
    criterion = nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

    # Train the Model
    train_data_loader = get_train_data_loader()
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_data_loader):
            images = Variable(images).to(device)
            labels = Variable(labels.float()).to(device)
            predict_labels = cnn(images)
            loss = criterion(predict_labels, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i + 1) % 10 == 0:
                print("epoch:", epoch, "step:", i, "loss:", loss.item())
            if (i + 1) % 100 == 0:
                torch.save(cnn.state_dict(), "./model.pkl")
                print("Model Saved")
        print("epoch:", epoch, "step:", i, "loss:", loss.item())
    torch.save(cnn.state_dict(), "./model.pkl")
    print("Last Model Saved")


main()

Cuda Available
Model Initialization
epoch: 0 step: 9 loss: 0.3802366256713867
epoch: 0 step: 19 loss: 0.3388024568557739
epoch: 0 step: 29 loss: 0.3370249569416046
epoch: 0 step: 39 loss: 0.33673155307769775
epoch: 0 step: 49 loss: 0.33487895131111145
epoch: 0 step: 59 loss: 0.3334570527076721
epoch: 0 step: 69 loss: 0.32869869470596313
epoch: 0 step: 79 loss: 0.32516375184059143
epoch: 0 step: 89 loss: 0.31953591108322144
epoch: 0 step: 99 loss: 0.3184945583343506
Model Saved
epoch: 0 step: 109 loss: 0.31147319078445435
epoch: 0 step: 119 loss: 0.3075438439846039
epoch: 0 step: 129 loss: 0.2915128767490387
epoch: 0 step: 139 loss: 0.28029128909111023
epoch: 0 step: 149 loss: 0.2822398543357849
epoch: 0 step: 159 loss: 0.2706435024738312
epoch: 0 step: 169 loss: 0.2694072723388672
epoch: 0 step: 179 loss: 0.2491394579410553
epoch: 0 step: 189 loss: 0.25083959102630615
epoch: 0 step: 199 loss: 0.24862685799598694
Model Saved
epoch: 0 step: 209 loss: 0.2509981393814087
epoch: 0 step: 219

In [0]:
import numpy as np
import torch
from torch.autograd import Variable


def main():
    # GPU Setting
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('Cuda Available')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    cnn = CNN()
    cnn.to(device)
    cnn.eval()
    cnn.load_state_dict(torch.load('model.pkl'))
    print("Model Loaded")

    data_loader = get_test_data_loader()

    correct = 0
    total = 0
    for i, (images, labels) in enumerate(data_loader):
        image = images
        variable = Variable(image).to(device)
        predict_label = cnn(variable).to(device)

        c0 = character_set[np.argmax(predict_label[0, 0:character_set_length].data.numpy())]
        c1 = character_set[np.argmax(predict_label[0, character_set_length:2 * character_set_length].data.numpy())]
        c2 = character_set[np.argmax(predict_label[0, 2 * character_set_length:3 * character_set_length].data.numpy())]
        c3 = character_set[np.argmax(predict_label[0, 3 * character_set_length:4 * character_set_length].data.numpy())]

        predict_label = '%s%s%s%s' % (c0, c1, c2, c3)
        true_label = decode(labels.numpy()[0])
        total += labels.size(0)

        if predict_label == true_label:
            correct += 1
        if total % 200 == 0:
            print('Test Accuracy of the model on the %d test images: %f %%' % (total, 100 * correct / total))

    print('Test Accuracy of the model on the %d test images: %f %%' % (total, 100 * correct / total))


main()

Model Loaded
Test Accuracy of the model on the 200 test images: 91.000000 %
Test Accuracy of the model on the 400 test images: 88.250000 %
Test Accuracy of the model on the 600 test images: 88.166667 %
Test Accuracy of the model on the 800 test images: 88.000000 %
Test Accuracy of the model on the 1000 test images: 88.800000 %
Test Accuracy of the model on the 1200 test images: 88.333333 %
Test Accuracy of the model on the 1400 test images: 88.357143 %
Test Accuracy of the model on the 1600 test images: 88.062500 %
Test Accuracy of the model on the 1800 test images: 87.777778 %
Test Accuracy of the model on the 2000 test images: 87.600000 %
Test Accuracy of the model on the 2200 test images: 87.636364 %
Test Accuracy of the model on the 2400 test images: 87.500000 %
Test Accuracy of the model on the 2600 test images: 87.384615 %
Test Accuracy of the model on the 2800 test images: 87.535714 %
Test Accuracy of the model on the 3000 test images: 87.533333 %
Test Accuracy of the model on t

In [0]:
# Model Download
from google.colab import files

files.download('model.pkl')

In [0]:
''' Adversarial Captcha Generator '''

from __future__ import print_function
import torch.nn as nn
import torch
from torch.autograd import Variable
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt


# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad
    # Adding clipping to maintain [0, 1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image


epsilon = 0.05
directory = 'adv'


def main():
    if os.path.exists(directory):
        shutil.rmtree(directory)
    if not os.path.exists(directory):
        os.makedirs(directory)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    cnn = CNN()
    cnn.to(device)
    cnn.eval()
    cnn.load_state_dict(torch.load('model.pkl'))
    print("Model Loaded")

    criterion = nn.MultiLabelSoftMarginLoss()
    data_loader = get_test_data_loader()

    correct = 0
    total = 0
    count = 1

    for i, (images, labels) in enumerate(data_loader):
        image = images
        variable = Variable(image).to(device)
        variable.requires_grad = True
        labels = Variable(labels).to(device)
        predict_label = cnn(variable)

        # plt.imshow(variable.squeeze().detach().cpu().numpy())
        # plt.show()

        loss = criterion(predict_label, labels.float())
        cnn.zero_grad()
        loss.backward()

        data_grad = variable.grad.data
        perturbed_image = fgsm_attack(variable, epsilon, data_grad)
        predict_label = cnn(perturbed_image)

        # numpy_image = perturbed_image.squeeze().detach().cpu().numpy()
        # plt.imshow(numpy_image)
        # plt.show()
        # fig = plt.gcf()
        # fig.savefig(directory + '/' + str(count) + '.png')
        count += 1

        c0 = character_set[np.argmax(predict_label[0, 0:character_set_length].data.cpu().numpy())]
        c1 = character_set[np.argmax(predict_label[0, character_set_length:2 * character_set_length].data.cpu().numpy())]
        c2 = character_set[np.argmax(predict_label[0, 2 * character_set_length:3 * character_set_length].data.cpu().numpy())]
        c3 = character_set[np.argmax(predict_label[0, 3 * character_set_length:4 * character_set_length].data.cpu().numpy())]

        predict_label = '%s%s%s%s' % (c0, c1, c2, c3)
        true_label = decode(labels.cpu().numpy()[0])
        total += labels.size(0)

        if predict_label == true_label:
            correct += 1
        if total % 20 == 0:
            print('Test Accuracy of the model on the %d test images: %f %%' % (total, 100 * correct / total))

    print('Test Accuracy of the model on the %d test images: %f %%' % (total, 100 * correct / total))


main()

Model Loaded
Test Accuracy of the model on the 20 test images: 40.000000 %
Test Accuracy of the model on the 40 test images: 45.000000 %
Test Accuracy of the model on the 60 test images: 40.000000 %
Test Accuracy of the model on the 80 test images: 35.000000 %
Test Accuracy of the model on the 100 test images: 32.000000 %
Test Accuracy of the model on the 120 test images: 31.666667 %
Test Accuracy of the model on the 140 test images: 34.285714 %
Test Accuracy of the model on the 160 test images: 34.375000 %
Test Accuracy of the model on the 180 test images: 32.777778 %
Test Accuracy of the model on the 200 test images: 32.000000 %
Test Accuracy of the model on the 220 test images: 33.181818 %
Test Accuracy of the model on the 240 test images: 32.916667 %
Test Accuracy of the model on the 260 test images: 32.692308 %
Test Accuracy of the model on the 280 test images: 31.785714 %
Test Accuracy of the model on the 300 test images: 30.666667 %
Test Accuracy of the model on the 320 test ima